# Data Handling demo

Information to provide forced-fit photometry for several ZTF sources has been produced (see [Demo.ipynb](https://github.com/yaoyuhan/ForcePhotZTF/blob/master/demo.ipynb)). This short notebook details how to extract the relevant information and then write the results from the MCMC fitting procedure back into the data files. 

In [2]:
import numpy as np
import pandas as pd
from astropy.table import Table
import glob

import matplotlib.pyplot as plt
%matplotlib notebook

In [9]:
info_path = "../early_Ia/2018/info/"
xy_path = "../early_Ia/2018/xydata/"

In [10]:
info_files = glob.glob(info_path + "force_phot_ZTF*fits")
xy_files = glob.glob(xy_path + "xydata_ZTF*fits")

In [21]:
# start with well known example
ztf_name = 'ZTF18aaxsioa'
info_file = info_path + 'force_phot_{}_info_refcut.fits'.format(ztf_name)
xy_file = xy_path + 'xydata_{}_refcut.fits'.format(ztf_name)

info_tbl = Table.read(info_file)
xy_tbl = Table.read(xy_file)
info_df = info_tbl.to_pandas()
xy_df = xy_tbl.to_pandas()

At this stage, operations are run on the data in the xy-table to measure the slope necessary to match the PSF to the difference image. This data is returned as a series of arrays with indicies based on the indicies in the xy data tables. 

The cell below includes example output from running the parallel processing.

In [46]:
output = [np.array([   0.        , -135.32920642,   62.96600296,    2.80687566,
           1.55688658]),
 np.array([ 1.00000000e+00, -2.26578183e+03,  1.09974331e+03,  1.41724513e+02,
         5.54310802e+00]),
 np.array([ 2.        ,  4.04412279, 83.4419023 , -0.16813332,  1.7397543 ]),
 np.array([  3.        ,  -9.8850982 ,  73.00987746, -11.69389273,
          2.09714441])]

output_arr = np.array(output)

In [51]:
# test that ordering is identical
if np.all(info_df['diffimgname'] == xy_df['path'].unique()):
    Fmcmc = np.zeros_like(info_df['zp'])
    Fmcmc_unc = np.zeros_like(Fmcmc)
    amcmc = np.zeros_like(Fmcmc)
    amcmc_unc = np.zeros_like(Fmcmc)
    for res_idx in output_arr[:,0].astype(int):
        idx = np.where(xy_df['index'].unique() == res_idx)[0]
        Fmcmc[idx] = output_arr[res_idx, 1]
        Fmcmc_unc[idx] = output_arr[res_idx, 2]
        amcmc[idx] = output_arr[res_idx, 3]
        amcmc_unc[idx] = output_arr[res_idx, 4]
else:
    raise ValueError("Input files do not have the same order")

In [55]:
F0 = 10**(info_df['zp'].values/2.5)
F0_unc = F0 / 2.5 * np.log(10) * info_df['ezp']
Fratio =  Fmcmc/F0
Fratio_unc = np.hypot(Fmcmc_unc/F0, Fmcmc*F0_unc/F0**2)

In [63]:
info_df['Fmcmc'] = Fmcmc
info_df['Fmcmc_unc'] = Fmcmc_unc
info_df['Fratio'] = Fratio
info_df['Fratio_unc'] = Fratio_unc

info_df.to_hdf(info_path + '{}_force_phot.h5'.format(ztf_name), 'lc')

The resulting hdf5 file now holds all the necessary information to create a force photometry light curve.